# Homework 6 is a bitch
For that reason I am walking through making a rnn to predict stock prices in a slow manner.

I will start with the absolute basics: 

## 1 (COMPLETE)
To start, I will be predicting AAPL stock only

I will only be looking at closing price

I will use the log values

I will predict price of the next day

## 1.1 (COMPLETE) - keras is easier so going with that
Implement the same thing in tensorflow to see what is easier


## 2 (COMPLETE)

After I prove that I can do that, I will predict for the 5 values

## 3 (COMPLETE)
After that I will use embedding of the stock symbol

## 4 
Make the network predict out 10 days on its own, use teacher forcing

## 5 
Add in all of the datasets for the all the stocks


# Data Preprocessing

In [ ]:
# IMPORTS

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable


In [ ]:
# import the csv of AAPL data

# bring in dataset with the dates as the index
df = pd.read_csv('stock_data/AAPL_data.csv')

df.head()

In [ ]:
# give some details of the dataframe
df.describe()

In [ ]:
# show a plot of the prices for some of the values

df['Close'].plot()
df['High'].plot()
df['Low'].plot()
df['Open'].plot()

plt.show()

## Modify the data
We are scaling the dat between -1 and 1

In [ ]:
scaler_close = MinMaxScaler(feature_range=(-1, 1))
scaler_open = MinMaxScaler(feature_range=(-1, 1))
scaler_high = MinMaxScaler(feature_range=(-1, 1))
scaler_low = MinMaxScaler(feature_range=(-1, 1))
scaler_volume = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
# TODO: fix the scaler - it is using only the volume fit as that was called last

aug_5_feat = df[['Close', 'Open', 'High', 'Low', 'Volume']]
aug_5_feat['Close'] = scaler_close.fit_transform(aug_5_feat['Close'].values.reshape(-1, 1))
aug_5_feat['Open'] = scaler_open.fit_transform(aug_5_feat['Open'].values.reshape(-1, 1))
aug_5_feat['High'] = scaler_high.fit_transform(aug_5_feat['High'].values.reshape(-1, 1))
aug_5_feat['Low'] = scaler_low.fit_transform(aug_5_feat['Low'].values.reshape(-1, 1))
aug_5_feat['Volume'] = scaler_volume.fit_transform(aug_5_feat['Volume'].values.reshape(-1, 1))
aug_5_feat.head()

## Split the data into train and prediction
We will be predicting the next day value

In [ ]:
aug_5_feat.head()

In [ ]:
def split_train_pred(df, w):
    # w is the size of the window
    df_raw = df.to_numpy()
    data = []
    
    for i in range(len(df) - w):
        data.append(df_raw[i:i+w+1])
    
    data = np.array(data)
    test_set_size = int(np.round(.2*data.shape[0]))
    train_set_size = data.shape[0] - test_set_size
    
    x = data[:train_set_size, :-1, :]
    y = data[:train_set_size, -1, :]
    
    x_test = data[train_set_size:, :-1]
    y_test = data[train_set_size:, -1, :]
        
    return x, y, x_test, y_test

In [ ]:
x_5, y_5, x_5_test, y_5_test = split_train_pred(aug_5_feat, 20)

In [ ]:
print(x_5.shape)
print(y_5.shape)

print(x_5[1])
print(y_5[0])

        

In [ ]:
print(x_5[0])
x_train_tf = np.array(x_5)
y_train_tf = np.array(y_5)
x_test_tf = np.array(x_5_test)
y_test_tf = np.array(y_5_test)

## Create Dict for stock -> int

In [ ]:
print(x_train_tf.shape)
stock_dict = {'AAPL': 1}

stock_input = []
for i in range(991):
    stock_input.append(stock_dict['AAPL'])


x_stock_train = np.array(stock_input).reshape(991, 1)
print(x_stock_train.shape)

# Build Model (Tensorflow)

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Lambda, Concatenate, Reshape
from tensorflow.keras import Model
import tensorflow

Actual model building (switch from sequential to functional)

In [ ]:
stock_inp_dim = 20
stock_inp_dim_2 = 5
num_stocks = 503
emb_dim = 16

def format_emb(x):
    new_list = []
    for i in x:
        interm = []
        for j in range(5):
            interm.append(i)
        new_list.append(interm)
    return np.array(new_list)

# Inputs for stock prices and for stock name int
inputA = Input(shape=(stock_inp_dim, stock_inp_dim_2)) # input is a sequence of stocks, with 5 features per day
inputB = Input(shape=(1)) # input is a single integer mapped to a stock

# Embedding the stock name and modifying to be able to cat with the sequence
x = Embedding(num_stocks, emb_dim)(inputB)
x = Dense(emb_dim*5, trainable=False)(x)
x = Reshape((emb_dim, 5))(x)

# Concatenate the price sequence with the embedded
combined = Concatenate(axis=1)([inputA, x])

# LSTM stuff
y = LSTM(4)(combined)
y = Dense(5)(y)

model = Model(inputs=[inputA, inputB], outputs=y)

model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
model.fit(x=[x_train_tf, x_stock_train], y=y_train_tf, epochs=50, batch_size=32, verbose=2)

In [ ]:
print(type(x_test_tf))
print(y_train_tf.shape)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

# make predictions
trainPredict = model.predict((x_train_tf, x_stock_train))
testPredict = model.predict((x_test_tf, x_stock_train[:248]))

# invert predictions
trainPredict = scaler_close.inverse_transform(trainPredict[:, 0].reshape(-1, 1))
trainY = scaler_close.inverse_transform(y_train_tf[:, 0].reshape(-1, 1))
testPredict = scaler_close.inverse_transform(testPredict[:, 0].reshape(-1, 1))
testY = scaler_close.inverse_transform(y_test_tf[:, 0].reshape(-1, 1))

In [ ]:
print(trainY.shape)
print(testY.shape)
print(trainPredict[0])

In [ ]:
look_back = 19

# shift train predictions for plotting
# trainPredictPlot = np.empty_like(aug_5_feat['Close'])
trainPredictPlot = np.empty(shape=(1259, 1))
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
# testPredictPlot = np.empty_like(aug_5_feat['Close'])
testPredictPlot = np.empty(shape=(1259, 1))
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)-19:len(aug_5_feat['Close'])-1, :] = testPredict
# testPredictPlot[len(trainPredict)+(look_back*2)+1:len(aug_df)-1, :] = testPredict
# plot baseline and predictions

original = np.concatenate((trainY, testY))

plt.plot(original, label='original')
plt.plot(trainPredictPlot, label='train pred')
plt.plot(testPredictPlot, label='test pred')
plt.legend()
plt.show()

